In [1]:
import os
import sys
os.chdir("../")
sys.path.append("../")

import re
import pickle
import numpy as np
import pandas as pd
import json
import logging
import torch
import math
import time
from tqdm import tqdm_notebook as tqdm
from datetime import datetime
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader
from collections import defaultdict
from transformers import AutoTokenizer, AutoModel
from utils.MIND import MIND, MIND_news, MIND_history, MIND_recall
from utils.Manager import Manager

logger = logging.getLogger(__name__)

In [2]:
# t = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir=config.path + "bert_cache/")
# t2 = AutoTokenizer.from_pretrained('microsoft/deberta-base', cache_dir=config.path + "bert_cache/")
# t3 = AutoTokenizer.from_pretrained('google/bigbird-roberta-base', cache_dir=config.path + "bert_cache/")

In [2]:
# config.reducer = 'bm25'
# config.reducer = 'bow'
# config.reducer = 'matching'
# config.reducer = 'first'

# config.signal_length = 10
config.scale = 'large'
config.impr_size = 2000
# config.mode = 'recall'
# config.news = None
# config.fast = True
# config.news = 21563

# config.bert = 'microsoft/deberta-base'
# config.embedding = 'deberta'

# config.bert = 'bert-base-uncased
# config.embedding = 'bert'

manager = Manager(config)
# file_directory = manager.path + "MIND/MIND{}_{}/".format(manager.scale, manager.mode)
file_directory = manager.path + "MIND/MIND{}_dev/".format(manager.scale)
# manager.gather_same_user_impr()

# a = MIND_history(manager, file_directory)
# b = MIND_news(manager)

In [3]:
b = MIND_news(manager, file_directory)

[2021-11-10 15:10:58,201] INFO (utils.MIND) process NO.0 loading cached user behavior from data/cache/MIND/behaviors/MINDlarge_dev/2000/behaviors.pkl
[2021-11-10 15:11:02,021] INFO (utils.MIND) process NO.0 loading cached news tokenization from data/cache/MIND/news/bert/MINDlarge_dev/news.pkl


In [5]:
b[1]

{'cdd_id': 1,
 'cdd_encoded_index': array([  101,  1996,  9639,  3035,  3870,  1010,  3159,  2798,  1010,
         1998,  3159,  5170,  8415,  2011,  4497,  1996, 14960,  2015,
         1010, 17764,  1010,  1998,  2062,  2008,  1996, 15426,  2064,
         1005,  1056,  2444,  2302,  1012,  9580, 13238,  9777,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0]),
 'cdd_attn_mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
  

In [8]:
behaviors = pickle.load(open('data/cache/bert/MINDlarge_dev/1000/behaviors.pkl','rb'))
# news = pickle.load(open('data/cache/bert/MINDdemo_train/news.pkl','rb'))
# news = pickle.load(open('data/cache/bert/MINDlarge_train/news.pkl','rb'))
# enc = news['encoded_news']

imprs = behaviors['imprs']
news_set = set()

for impr in imprs:
    news_set.update(impr[1])

In [15]:
torch.save(torch.tensor(list(news_set)), "data/recall/news.pt")

In [ ]:
# bm25, entity, first
index = 1

# news = t.convert_ids_to_tokens(a[index]['his_encoded_index'][1])
# mask = a[index]['his_attn_mask'][1]
# word = manager.convert_tokens_to_words(news)
# subword = a[index]['his_subword_index'][1]

# for i,j,k in zip(news,mask,subword):
#     print(i,j,k)
res = a[index]
ids = res['cdd_id'][1]
news = t.convert_ids_to_tokens(res['cdd_encoded_index'][1])
mask = res['cdd_attn_mask'][1]
# word = manager.convert_tokens_to_words(news)
# subword = res['cdd_subword_index'][1]
print(ids)
for i,j,k in zip(news,mask):
    print(i,j,k)

In [ ]:
news = t.convert_ids_to_tokens(a[0]['his_encoded_index'][1])
mask = a[0]['his_attn_mask'][1]
word = manager.convert_tokens_to_words(news)
# subword = a[0]['his_subword_index'][1]
dedup = a[0]['his_refined_mask'][1]

for i,j,k in zip(news,mask,dedup):
    print(i,j,k)